In [7]:
%load_ext autoreload
%autoreload 2
    
import yaml
from os import listdir
from os.path import isfile, join
from detector_model import get_model
import scipy
import numpy as np
from keras_tqdm import TQDMNotebookCallback
import matplotlib.pyplot as plt
from permutator import permutate
from toolbox.profiler import time_function

num_classes = 3 #Red, Yellow, Green
label_mappings = {
    'Red': 0,
    'Yellow': 1,
    'Green': 2
}  

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
def build_result_mask(size, annotations, scale=1.0):
    mask = np.zeros((int(size[0]*scale), int(size[1]*scale), 1), dtype=np.uint8)
    for annotation in annotations:
        xmin = int(annotation['xmin'] * scale)
        xmax = int((annotation['xmin'] + annotation['x_width']) * scale)
        ymin = int(annotation['ymin'] * scale)
        ymax = int((annotation['ymin'] + annotation['y_height']) * scale)
        layer_number = label_mappings[annotation['class']]
        
        mask[ymin:ymax, xmin:xmax] = 1
    return mask

def get_annotations(path):
    annotations = yaml.load(open(path, 'rb'))
    return {x['filename'].split('/')[1]: x['annotations'] for x in annotations}

def get_file(path, annotation):
    scale = 0.8
    base_image = scipy.misc.imread(path)
    image = scipy.misc.imresize(base_image, scale)
    #base_image = (base_image / 255) - 0.5
    mask = build_result_mask(base_image.shape, annotation, scale=scale)    
    return image, mask

def get_sim_training_data():
    annotations = get_annotations('data/sim_training_data/sim_data_annotations.yaml')
    file_location = 'data/sim_training_data/sim_data_capture/'
    all_files = [(f, join(file_location, f)) for f in listdir(file_location) if isfile(join(file_location, f))]
    return [get_file(join(file_location, file_name), file_annotations) for file_name, file_annotations in annotations.items()]

def show_image(*imgs, title=''):
    if len(imgs) > 1:
        img = np.concatenate(tuple([x for x in imgs]), axis=1)
    else:
        img = imgs[0]
    fig = plt.figure(figsize=(10,100))
    plt.imshow(img)
    fig.suptitle(title)

dataset = get_sim_training_data()

/home/jamesaustin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/jamesaustin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


In [9]:
np.save('./test_label', dataset[0][1])
scipy.misc.imsave('./test_image.png', dataset[0][0])

/home/jamesaustin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import random
import scipy

dataset = shuffle(dataset)
training_set, test_set = train_test_split(dataset, test_size=0.3)
training_set, validation_set = train_test_split(training_set, test_size=0.1)

def myGenerator(dataset, permutate_amount=0.0):
    #loading data
    while True:
        for image, label in dataset:
            if permutate_amount > random.random():
                image, label = permutate(image, label)

            result = [np.array([image]), np.array([label])]
            yield result
training_generator = myGenerator(training_set, permutate_amount=0.5)
validating_generator = myGenerator(validation_set)



In [12]:
import shutil
import os

def clear_stored_images():
    try:
        shutil.rmtree('./test_data/')
    except Exception as e:
        pass
    os.makedirs('./test_data/')


In [18]:
from keras.callbacks import ModelCheckpoint, Callback
checkpointer = ModelCheckpoint(filepath="./tmp/weights__5.{epoch:02d}-{val_loss:.2f}.hdf5", verbose=1)
clear_stored_images()
model = get_model(training_set[0][0].shape[0],training_set[0][0].shape[1])
@time_function()
def get_prediction(model, image):
    predicted_mask = model.predict(np.array([image]))[0]
    return np.resize(predicted_mask, (predicted_mask.shape[0], predicted_mask.shape[1]))

base_image = np.concatenate((validation_set[15][0],
                             validation_set[16][0],
                             validation_set[17][0],
                             validation_set[18][0],
                             validation_set[19][0],
                             validation_set[12][0],
                             validation_set[13][0],
                             validation_set[14][0],), axis=1)

scipy.misc.imsave(f"./test_data/base_image.jpg", base_image)


class other_callback(Callback):
    
    def on_epoch_end(*args, **kwargs):
        i1 = get_prediction(model, validation_set[15][0])
        i2 = get_prediction(model, validation_set[16][0])
        i3 = get_prediction(model, validation_set[17][0])
        i4 = get_prediction(model, validation_set[18][0])
        i5 = get_prediction(model, validation_set[19][0])
        i6 = get_prediction(model, validation_set[12][0])
        i7 = get_prediction(model, validation_set[13][0])
        i8 = get_prediction(model, validation_set[14][0])

        image = np.concatenate((i1,i2,i3,i4,i5,i6,i7,i8), axis=1)
        
        scipy.misc.imsave(f"./test_data/E{str(args[1]).zfill(5)}_{args[2]['val_dice_coef']:.2f}.jpg", image)
#         validation = validation_set[15][0]
#         predicted_mask = model.predict(np.array([validation]))[0]
#         show_image(np.resize(predicted_mask, (predicted_mask.shape[0], predicted_mask.shape[1])))
        
        
        #import pdb; pdb.set_trace()
        pass
model.fit_generator(training_generator, 
                    steps_per_epoch=len(training_set), 
                    epochs=100, 
                    verbose=0,
                    validation_data=validating_generator, 
                    validation_steps=len(validation_set), 
                    callbacks=[checkpointer,TQDMNotebookCallback(),other_callback()])

model.save_weights('./fast_traffic_detection_model_v2.hdf5')

/home/jamesaustin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


HBox(children=(IntProgress(value=0, description='Training'), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 0', max=173), HTML(value='')))

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/jamesaustin/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/jamesaustin/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py", line 63, in run
    for instance in self.tqdm_cls._instances:
  File "/home/jamesaustin/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration




Epoch 00001: saving model to ./tmp/weights__5.01--0.30.hdf5
Function (get_prediction) Finished: 0.166674s
Function (get_prediction) Finished: 0.019436s
Function (get_prediction) Finished: 0.018045s
Function (get_prediction) Finished: 0.019173s
Function (get_prediction) Finished: 0.018274s
Function (get_prediction) Finished: 0.020343s
Function (get_prediction) Finished: 0.017138s
Function (get_prediction) Finished: 0.019788s


...


Epoch 00098: saving model to ./tmp/weights__5.98--0.83.hdf5
Function (get_prediction) Finished: 0.020404s
Function (get_prediction) Finished: 0.019179s
Function (get_prediction) Finished: 0.019042s
Function (get_prediction) Finished: 0.017926s
Function (get_prediction) Finished: 0.018544s
Function (get_prediction) Finished: 0.021246s
Function (get_prediction) Finished: 0.017954s
Function (get_prediction) Finished: 0.020327s


HBox(children=(IntProgress(value=0, description='Epoch 98', max=173), HTML(value='')))


Epoch 00099: saving model to ./tmp/weights__5.99--0.89.hdf5
Function (get_prediction) Finished: 0.020136s
Function (get_prediction) Finished: 0.02098s
Function (get_prediction) Finished: 0.020053s
Function (get_prediction) Finished: 0.018524s
Function (get_prediction) Finished: 0.019087s
Function (get_prediction) Finished: 0.023467s
Function (get_prediction) Finished: 0.020101s
Function (get_prediction) Finished: 0.023768s


HBox(children=(IntProgress(value=0, description='Epoch 99', max=173), HTML(value='')))


Epoch 00100: saving model to ./tmp/weights__5.100--0.90.hdf5
Function (get_prediction) Finished: 0.020895s
Function (get_prediction) Finished: 0.025588s
Function (get_prediction) Finished: 0.01988s
Function (get_prediction) Finished: 0.019112s
Function (get_prediction) Finished: 0.017702s
Function (get_prediction) Finished: 0.020381s
Function (get_prediction) Finished: 0.021667s
Function (get_prediction) Finished: 0.023171s
